In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn import RNNCell

In [4]:
SOS_token = 0
EOS_token = 4
vocab_size = 5
x_data = np.array([[SOS_token, 3, 3, 2, 3, 2],[SOS_token, 3, 1, 2, 3, 1],[SOS_token, 1, 3, 2, 2, 1]], dtype=np.int32)

print("data shape: ", x_data.shape)
sess = tf.InteractiveSession()

output_dim = vocab_size
batch_size = len(x_data)
seq_length = x_data.shape[1]
embedding_dim = 2

init = np.arange(vocab_size*embedding_dim).reshape(vocab_size,-1)

train_mode = True
alignment_history_flag = False

data shape:  (3, 6)


In [26]:
class MyBasicRNNWrapper(RNNCell):
    # property(output_size, state_size) 2개와 call을 정의하면 된다.
    def __init__(self,state_dim,name=None):
        super(MyBasicRNNWrapper, self).__init__(name=name)
        self.sate_size = state_dim

    @property
    def output_size(self):
        return self.sate_size  

    @property
    def state_size(self):
        return self.sate_size  

    def build(self, inputs_shape):
        # 필요한 trainable variable이 있으면 여기서 생성하고, self.built = True로 설정하면 된다.
        input_depth = inputs_shape[1].value
        self._kernel = tf.get_variable('kernel', shape=[input_depth + self.sate_size, self.sate_size])
        self._bias = tf.get_variable('bias', shape=[self.sate_size],  initializer=tf.zeros_initializer(dtype=tf.float32))
    
        self.built = True  # 필요한 변수가 선언되었다는 것을 super에게 알려주는 역할.

    def call(self, inputs, state):
        # 이 call 함수를 통해 cell과 cell이 연결된다.
        inputs = tf.layers.dense(inputs,units=self.input_depth,kernel_initializer = tf.contrib.layers.xavier_initializer(False), activation=tf.nn.relu)
        gate_inputs = tf.matmul(tf.concat((inputs,state),axis=1),self._kernel)
        gate_inputs = tf.nn.bias_add(gate_inputs,self._bias)
        cell_output = tf.tanh(gate_inputs)
        next_state = cell_output
        return cell_output, next_state 

In [38]:
X = tf.placeholder(tf.float32, [None, 6])
Y = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.random_normal([2, 1]))
b = tf.Variable(tf.random_normal([1]))

cell = LSTMcell()
cell.build([3,6])
outputs, states = cell.call(cell, X)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train = tf.train.AdamOptimizer(0.0001).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


AttributeError: 'int' object has no attribute 'value'

In [31]:

for e in range(1000):
    _, _cost = sess.run([train, cost],feed_dict={X: batch_xs, Y: batch_ys})
    test_batch_size = len(mnist.test.images)
    a = sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(test_batch_size, 28, 28), Y:mnist.test.labels})
    print(a)

NameError: name 'cost' is not defined